# HISTORICO OP

In [26]:
import pandas as pd
import numpy as np

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)

In [27]:
import xmlrpc.client
import pandas as pd
import re

# --- Configuración de conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# --- Work orders asociadas (solo desde 2023-01-01) ---
workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line.flab", "search_read",
    [
        [
            ("fecha_inicial", ">=", "2023-01-01")
        ],
        [
           "workcenter_id", "cantidad_fabricada", "display_name", "operaion",
            "production_id", "product_reference", "delay", "fecha_final", "state", "fecha_inicial"
        ]
    ]
)


df_workorders = pd.DataFrame(workorders)

# --- Limpieza y extracción de campos ---
df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["nombre"] = df_workorders["operaion"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["centro_produccion"] = df_workorders["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["maquina_real"] = df_workorders["display_name"].str.split(" - ", n=1).str[0].str.strip()


# Patrón: "DA" + dígitos + (opcional) letras (cualquiera mayúsc/minúsc)
pattern = r'(?i)(DA[C]?\d+[A-Z]{0,4})'



# Opción A: usando extract (rápido, pero a veces \b puede fallar con puntuación)
df_workorders["nom_produccion"] = df_workorders["display_name"].str.extract(pattern, expand=False)

df_workorders = df_workorders.drop(columns=[
    "operaion", "workcenter_id", "production_id", "product_reference"
])

#tarda 1 min

In [29]:
df_workorders["state"].unique()

array(['done', 'cancel', 'pause', 'startworking'], dtype=object)

In [72]:
import pandas as pd

# --- Crear columna tasa (por fila) ---
df_workorders["tasa_por_unidad"] = df_workorders["delay"] / df_workorders["cantidad_fabricada"]

# --- Asegurar que la fecha_final sea tipo datetime ---
df_workorders["fecha_final"] = pd.to_datetime(df_workorders["fecha_final"], errors="coerce")

df_workorders["fecha_inicial"] = pd.to_datetime(df_workorders["fecha_inicial"], errors="coerce")



# --- Agrupar por producto, centro y máquina ---
df_resumen = (
    df_workorders
    .groupby(["nom_produccion", "centro_produccion", "maquina_real"], as_index=False)
    .agg(
        cantidad_total=("cantidad_fabricada", "sum"),
        delay_total=("delay", "sum"),
        tasa_promedio=("tasa_por_unidad", "mean"),
        n_datos=("tasa_por_unidad", "count"),
        ultima_fecha_produccion=("fecha_final", "max"),
        primera_fecha=("fecha_inicial","min")
    )
)


In [73]:
df_resumen["nom_produccion"].nunique()

672

In [74]:
df_resumen[df_resumen["nom_produccion"].str.contains("DA8041")]

,nom_produccion,centro_produccion,maquina_real,cantidad_total,delay_total,tasa_promedio,n_datos,ultima_fecha_produccion,primera_fecha
4678,DA8041,CORTE,ENTUBADORA 2,430.0,40053.0,100.819583,8,2025-09-12 12:06:34,2023-02-15 11:02:40
4679,DA8041,CORTE,ENTUBADORA 3,600.0,42125.0,72.280972,12,2025-10-02 10:11:54,2023-02-15 10:17:54
4680,DA8041,CORTE,REPUJADO,430.0,56890.0,132.639375,8,2025-10-03 10:02:01,2023-02-16 11:18:44
4681,DA8041,CORTE,TAPAS,630.0,45328.0,71.105694,12,2025-09-18 14:26:30,2023-02-15 08:47:24
4682,DA8041,EMBALAJE,BOLSA,40.0,767.0,19.175000,1,2025-10-06 11:23:42,2025-10-06 11:10:55
4683,DA8041,EMBALAJE,CONTROL DE CALIDAD,490.0,19387.0,38.416667,9,2025-10-06 11:01:45,2023-02-18 09:21:15
4684,DA8041,EMBALAJE,EMBALAJE,480.0,22179.0,43.618333,9,2025-10-06 11:23:37,2023-02-18 09:37:08
4685,DA8041,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,640.0,57625.0,92.549861,12,2025-10-06 10:09:24,2023-02-18 06:55:39
4686,DA8041,INYECCION,INYECCION,940.0,162120.0,171.738824,17,2025-01-23 09:41:25,2023-02-15 13:06:26
4687,DA8041,INYECCION,INYECCION NARANJA,80.0,7754.0,96.925000,2,2025-10-06 08:28:23,2025-10-03 14:06:56


# OP EN PROCESO Y CUANTO VA POR TASA

In [37]:
import xmlrpc.client
import pandas as pd
import re

# --- Configuración de conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Work orders en proceso (no terminadas) desde 2023-01-01 ---
workorders_en_proceso = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line.flab", "search_read",
    [
        [
            ("fecha_inicial", ">=", "2023-01-01"),
            ("state", "!=", ["done","cancel"])   # 👈 excluye las terminadas
        ],
        [
            "workcenter_id", "cantidad_fabricada", "display_name", "operaion",
            "production_id", "product_reference", "delay", "fecha_final", "state", "fecha_inicial"
        ]
    ]
)

df_en_proceso = pd.DataFrame(workorders_en_proceso)

# --- Limpieza y extracción de campos ---
df_en_proceso["OP"] = df_en_proceso["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_en_proceso["nombre"] = df_en_proceso["operaion"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_en_proceso["centro_produccion"] = df_en_proceso["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_en_proceso["maquina_real"] = df_en_proceso["display_name"].str.split(" - ", n=1).str[0].str.strip()
df_en_proceso["nom_produccion1"] = df_en_proceso["display_name"].str.split(" - ", n=1).str[1]
df_en_proceso["nom_produccion"] = df_en_proceso["nom_produccion1"].str.extract(r"\b(DA\d+[A-Za-z]?)\b")

df_en_proceso = df_en_proceso.drop(columns=[
    "operaion", "workcenter_id", "production_id", "product_reference", "nom_produccion1"
])



In [39]:
df_en_proceso.sort_values(by=["fecha_inicial"]).head(40)

,delay,fecha_inicial,display_name,fecha_final,cantidad_fabricada,state,id,OP,nombre,centro_produccion,maquina_real,nom_produccion
0,0.0,2025-10-01 06:46:30,"MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA9027 FILTRO AIRE 2o. ATLAS COPCO, GROVE, DEMAG",False,100.0,pause,112410,MO13185,ERIKA YULIEDT UMAÑA SABOGAL,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA9027
1,0.0,2025-10-03 14:57:39,"OMS - DA9027 FILTRO AIRE 2o. ATLAS COPCO, GROVE, DEMAG",False,100.0,pause,112714,MO13185,MARIA ANGELICA HERRERA LOPEZ,INYECCION,OMS,DA9027
2,0.0,2025-10-06 13:36:09,TAPAS Y MALLA - DA6001 FILTROAIRE PANEL CAT 6L4714 PANAL CUADRADO,False,30.0,pause,112789,MO13297,ALVARO MELO PRADA,CORTE,TAPAS Y MALLA,DA6001
3,0.0,2025-10-08 09:19:09,TECNONEXT NARANJA - DA2523 FILTRO AIRE CONICO FREIGHTLINER,False,80.0,pause,112933,MO13194,YEISON VILLANUEVA MELO,INYECCION,TECNONEXT NARANJA,DA2523
4,0.0,2025-10-08 09:19:30,TECNONEXT NARANJA - DA2523 FILTRO AIRE CONICO FREIGHTLINER,False,80.0,pause,112934,MO13194,MARIA PAULA PARRA PINILLA,INYECCION,TECNONEXT NARANJA,DA2523
5,0.0,2025-10-10 10:34:43,REFUERZO - DA6001 FILTROAIRE PANEL CAT 6L4714 PANAL CUADRADO,False,30.0,pause,113181,MO13297,ALVARO MELO PRADA,CORTE,REFUERZO,DA6001
6,0.0,2025-10-15 06:44:16,"OMS - DA9027 FILTRO AIRE 2o. ATLAS COPCO, GROVE, DEMAG",False,100.0,pause,113335,MO13185,BLANCA CECILIA AVELLA ARIAS,INYECCION,OMS,DA9027
7,0.0,2025-10-15 06:44:43,"OMS - DA9027 FILTRO AIRE 2o. ATLAS COPCO, GROVE, DEMAG",False,100.0,pause,113336,MO13185,JAVIER ESTEBAN GONZALEZ ARENAS,INYECCION,OMS,DA9027
8,0.0,2025-10-15 08:19:57,TECNONEXT NARANJA - DA3144 FILTRO AIRE 1o.,False,20.0,pause,113355,MO13341,MARIA PAULA PARRA PINILLA,INYECCION,TECNONEXT NARANJA,DA3144
9,0.0,2025-10-15 08:20:18,TECNONEXT NARANJA - DA3144 FILTRO AIRE 1o.,False,20.0,pause,113356,MO13341,YEISON VILLANUEVA MELO,INYECCION,TECNONEXT NARANJA,DA3144


# PARETO PRODUCCION

In [32]:
ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")